In [ ]:
DOWNLOAD_DIR = "data"
!wget -nc -P $DOWNLOAD_DIR "https://archive.ics.uci.edu/ml/machine-learning-databases/00240/UCI HAR Dataset.zip"
!wget -nc -P $DOWNLOAD_DIR "https://archive.ics.uci.edu/ml/machine-learning-databases/00240/UCI HAR Dataset.names"
!unzip -oq -d $DOWNLOAD_DIR "$DOWNLOAD_DIR/UCI HAR Dataset.zip"

In [ ]:
import glob
import os
import numpy as np
from keras.utils.np_utils import to_categorical

DATA_DIR = os.path.join(DOWNLOAD_DIR, "UCI HAR Dataset")

X_train_list = [np.loadtxt(f) for f in glob.glob(os.path.join(DATA_DIR, "train", "Inertial Signals", "*"))]
X_train = np.array(X_train_list).transpose((1, 0, 2))
y_train = np.loadtxt(os.path.join(DATA_DIR, "train", "y_train.txt")) - 1
Y_train = to_categorical(y_train)
X_test_list = [np.loadtxt(f) for f in glob.glob(os.path.join(DATA_DIR, "test", "Inertial Signals", "*"))]
X_test = np.array(X_test_list).transpose((1, 0, 2))
y_test = np.loadtxt(os.path.join(DATA_DIR, "test", "y_test.txt")) - 1
Y_test = to_categorical(y_test)

In [ ]:
from keras.callbacks import Callback
from IPython.core.display import clear_output

class ClearOutput(Callback):
    def on_epoch_end(self, epoch, logs={}):
        clear_output()

In [ ]:
from keras import backend as K
from keras.layers import Input, Dense, LSTM, Dropout
from keras.regularizers import l2
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import ProgbarLogger, TensorBoard

n_hidden = 32
lr = 0.0025
lambda_l2 = 0.0015
nb_epoch = 300
batch_size = 1500

input_shape = X_train.shape[1:]
output_shape = Y_train.shape[1]

model = Sequential()
model.add(LSTM(n_hidden, input_shape=input_shape, W_regularizer=l2(lambda_l2), return_sequences=True))
model.add(LSTM(n_hidden, W_regularizer=l2(lambda_l2)))
model.add(Dense(output_shape, activation='softmax'))
model.compile(optimizer=Adam(lr=lr), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, Y_train, nb_epoch=nb_epoch, batch_size=batch_size,
                    callbacks=[ProgbarLogger(), TensorBoard(log_dir='log'), ClearOutput()])

In [ ]:
print("LSTM test accuracy: {}".format(np.mean(np.argmax(model.predict(X_test), axis=1) == y_test)))

In [ ]:
%%time
import cesium as cs

features_to_use = [f for f in cs.features.GENERAL_FEATS if 'period' not in f and 'flux' not in f]
fset_train = cs.featurize.featurize_time_series(None, [X_i for X_i in X_train], None, 
                                                features_to_use=features_to_use,
                                                targets=y_train)

In [ ]:
%%time
from sklearn.ensemble import RandomForestClassifier

param_grid = {'n_estimators': [256, 512],
              'criterion': ['mse', 'mae'],
              'max_features': [0.165, 0.28, 0.33, 0.38, 0.43, 0.5, 0.667]
              }

cs_model = cs.build_model.build_model_from_featureset(fset_train, model_type='RandomForestClassifier',
                                                      params_to_optimize=param_grid)

In [ ]:
%%time
fset_test = cs.featurize.featurize_time_series(None, [X_i for X_i in X_test], None, 
                                               features_to_use=features_to_use,
                                               targets=y_test)
cs_pred = cs.predict.model_predictions(fset_test, cs_model, return_probs=False)
print("Cesium test accuracy: {}".format(np.mean(cs_pred.prediction == y_test)))

In [ ]:
print("Cesium test accuracy: {}".format(np.mean(cs_pred.prediction.values == y_test)))